# 加载预训练模型

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model_id = "mistralai/Mistral-7B-v0.1"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config = bnb_config,
    device_map = "auto",
    use_cache = False
    )
base_model.config.pretraining_tp = 1
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id) 
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 合并LoRA权重

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-7b-style/")

In [3]:
ft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer):

# 随机选择样本

In [4]:
from datasets import load_dataset

dataset = load_dataset("neuralwork/fashion-style-instruct")

In [5]:
from random import seed, randrange

# 创建一个新的格式函数
def format_instruction_new(sample) :
    return f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
{sample["input"]}
### Context:
{sample["context" ]}
### Response:
"""

# 设置种子
seed(42)

# randrange函数生成随机数
sample = dataset["train"][randrange(len(dataset['train']))]

# 创建推理prompt
prompt = format_instruction_new(sample) 
print(prompt)

You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
I'm a short, stocky woman with muscular calves and thighs. I like showing off my strong legs but need to balance my proportions on top.
### Context:
I'm going to a bachelorette / bachelor party.
### Response:



# 推理

In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Using device:", device)

# tokenize input text
input_ids = eval_tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

# inference, 5 outfit combinations make up around 700-750 tokens 
with torch.inference_mode():
    outputs = ft_model.generate(
        input_ids=input_ids, 
        max_new_tokens=800,
        do_sample=True,  
        top_p=0.9,
        temperature=0.9
    )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [7]:
input_ids

tensor([[    1,   995,   460,   264,  3327, 10176,  1703,  4626,  2570,  8844,
          7478,   304, 13278, 26350, 28723,  5938,   272,  1008,  2187,   304,
          3238,  5436,  3624, 28725,  9837,   395,   272,   317,  2035,  5397,
           297,   272,  2758,   298,  8270, 28705, 28782,  1008, 28733,  1399,
          1738,   304,  4160, 24842, 26350, 28723,    13, 27332, 11232, 28747,
            13, 28737, 28742, 28719,   264,  2485, 28725,  5756, 28724,  2971,
           395,  2009, 12723,  2300,  1855,   304, 21745, 28713, 28723,   315,
           737,  8102,   805,   586,  2967,  7969,   562,   927,   298,  7873,
           586, 12687,   594,   356,  1830, 28723,    13, 27332, 14268, 28747,
            13, 28737, 28742, 28719,  1404,   298,   264,   287, 14047, 11007,
           424,   732,   287, 18753,  4150, 28723,    13, 27332, 12107, 28747,
            13]])

In [8]:
len(input_ids[0])

111

In [9]:
outputs

tensor([[    1,   995,   460,   264,  3327, 10176,  1703,  4626,  2570,  8844,
          7478,   304, 13278, 26350, 28723,  5938,   272,  1008,  2187,   304,
          3238,  5436,  3624, 28725,  9837,   395,   272,   317,  2035,  5397,
           297,   272,  2758,   298,  8270, 28705, 28782,  1008, 28733,  1399,
          1738,   304,  4160, 24842, 26350, 28723,    13, 27332, 11232, 28747,
            13, 28737, 28742, 28719,   264,  2485, 28725,  5756, 28724,  2971,
           395,  2009, 12723,  2300,  1855,   304, 21745, 28713, 28723,   315,
           737,  8102,   805,   586,  2967,  7969,   562,   927,   298,  7873,
           586, 12687,   594,   356,  1830, 28723,    13, 27332, 14268, 28747,
            13, 28737, 28742, 28719,  1404,   298,   264,   287, 14047, 11007,
           424,   732,   287, 18753,  4150, 28723,    13, 27332, 12107, 28747,
            13,  2919,  7785, 19422,  2235, 28705, 28740, 28747,    13, 28733,
          6611, 28747,   330, 21163, 28725,  7767,  

In [10]:
len(outputs[0])

746

In [11]:
# decode token ids to text
outputs = outputs.detach().cpu().numpy()

outputs = eval_tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [12]:
outputs

["You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.\n### Input:\nI'm a short, stocky woman with muscular calves and thighs. I like showing off my strong legs but need to balance my proportions on top.\n### Context:\nI'm going to a bachelorette / bachelor party.\n### Response:\nOutfit Combination 1:\n- Top: A fitted, sleeveless crop top in a vibrant color like fuchsia or royal blue. This will draw attention to your upper body.\n- Bottom: High-waisted black skinny jeans to accentuate your waist and create a streamlined look.\n- Shoe: Black ankle strap heels to elongate your legs and add a touch of elegance.\n- Accessories: Pair the outfit with some dangling earrings and a statement clutch in a complementary color.\n\nOutfit Combination 2:\n- Top: A flowy, off-the-shoulder blouse in a fun pattern or b

In [13]:
output = outputs[0][len(prompt):]

print(f"Instruction: \n{sample['input']}\n") 
print(f"Context: \n{sample['context']}\n" ) 
print(f"Ground truth: \n{sample['completion']}\n") 
print(f"Generated output: \n{output}\n\n\n")

Instruction: 
I'm a short, stocky woman with muscular calves and thighs. I like showing off my strong legs but need to balance my proportions on top.

Context: 
I'm going to a bachelorette / bachelor party.

Ground truth: 
Outfit Combination 1:
- Top: A flowy, off-the-shoulder blouse in a bold color like royal blue or hot pink. The off-the-shoulder style will draw attention upward and balance your proportions.
- Bottom: High-waisted, black leather leggings. The high-waisted cut will emphasize your waistline and flatter your figure, while the leather material adds a touch of edge to the outfit.
- Shoe: Pair the outfit with black ankle boots with a chunky heel. The boots will elongate your legs while adding stability and comfort for a night of dancing.
- Accessories: Add a statement necklace with colorful gemstones and a stack of silver bangle bracelets to further enhance the look.

Outfit Combination 2:
- Top: A fitted, black bodysuit with a plunging V-neckline. The fitted style will ac